In [1]:
import numpy as np
import numpy.random as npr
import random
import matplotlib.pyplot as plt
from IPython.display import display, Audio
import torch as torch
import torchvision as tv
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio.transforms as audio
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split, KFold
import sys
from scipy.signal import resample
from PIL import Image

/home/lukasherron/.local/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [78]:
data_training = np.load('data_extra_1.npy', allow_pickle=True)

In [79]:
r = len(data_training)
for i in range(r):
    data_training[i] = resample(data_training[i], 100000)
    data_training[i] = (data_training[i] - np.average(data_training[i]))/max(abs(data_training[i]) + 1e-9)


In [80]:
spectrograms_1 = []
spectrograms_2 = []
spectrograms_3 = []
for i in range(r):
    
    sig = torch.Tensor(data_training[i])
    sg = audio.MelSpectrogram( 
                            sample_rate = 44000, n_fft=4096, 
                            win_length = 4096, hop_length=1024,
                            n_mels=128)(sig)
    sg = sg.detach().numpy()
    sg = np.array(transforms.Resize((128, 256))(Image.fromarray(sg)))
    spectrograms_1.append(np.log(sg + 1e-9))


    sg = audio.MelSpectrogram( 
                            sample_rate = 44000, n_fft=4096, 
                            win_length = 2048, hop_length=512,
                            n_mels=128)(sig)
    sg = sg.detach().numpy()
    sg = np.array(transforms.Resize((128, 256))(Image.fromarray(sg)))
    spectrograms_2.append(np.log(sg + 1e-9))


    sg = audio.MelSpectrogram( 
                            sample_rate = 44000, n_fft=4096, 
                            win_length = 1024, hop_length=256,
                            n_mels=128)(sig)
    sg = sg.detach().numpy()
    sg = np.array(transforms.Resize((128, 256))(Image.fromarray(sg)))
    spectrograms_3.append(np.log(sg + 1e-9))


features_log = np.stack([[spectrograms_1[i], spectrograms_2[i], spectrograms_3[i]] for i in range(r)])

In [81]:
# Sphering the spectrograms:
ZCA_spectrograms_1 = []
ZCA_spectrograms_2 = []
ZCA_spectrograms_3 = []

for i in range(r):
    for j in range(3):
        demeaned = np.array(features_log[i][j] - np.mean(features_log[i][j], axis=0))
        (r_prime,c) = demeaned.shape
        lam = 10**-7
        P = float(c)
        cov = 1/P*np.dot(demeaned, demeaned.T) + lam*np.eye(r_prime)
        D,V = np.linalg.eigh(cov)
        ZCAed = np.dot(V, np.dot(V.T, demeaned))
        if j == 0:
            ZCA_spectrograms_1.append(ZCAed)
        if j == 1:
            ZCA_spectrograms_2.append(ZCAed)
        if j == 2:
            ZCA_spectrograms_3.append(ZCAed)

ZCA_features_log = np.stack([[ZCA_spectrograms_1[i], ZCA_spectrograms_2[i], ZCA_spectrograms_3[i]] for i in range(r)])

In [84]:
np.save('extra_ZCA_features_1.npy', ZCA_features_log)